In [2]:
# 本次模型训练任务为根据多个特征属性预测房价
from keras.datasets import boston_housing
# 导入训练数据和测试数据
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
print(train_data.shape)
print(test_data.shape)

57344/57026 [==============================] - 1s 26us/step
(404, 13)
(102, 13)


In [9]:
"""
以上输出结果表明训练数据集有404条，测试数据集有102条。而每条数据集都包含13个方面的特征，它包括以下几种数据信息： 
1，城镇中非商业用地的比率 
2，每块面积25000英尺的地皮上，住宅用地的比率。 
3，城镇中非零售商业用地面积 
4，是否沿河，1表示沿河，0表示不沿河 
5，一氧化氮含量 
6，住宅的平均单间数 
7，1940年前自有产权房的数量 
8，住宅与波士顿五个工作区的距离 
9，接入高速公路的便利指数 
10，每一万美金所要缴纳的全额房产税 
11，城镇中的师生比 
12，1000*(BK - 0.63)^2, BK表示黑人比率 
13，底层人口的百分比
我们要预测的是自有产权房的中位价格，单位是千美元
"""
# 数据规范化
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis = 0)
train_data /= std
test_data -= mean
test_data /= std
"""
Keras 是为人类而非机器设计的 API。Keras 遵循减少认知困难的最佳实践: 
它提供一致且简单的 API，它将常见用例所需的用户操作数量降至最低，并且在用户错误时提供清晰和可操作的反馈。
这使 Keras 易于学习和使用。作为 Keras 用户，你的工作效率更高，能够比竞争对手更快地尝试更多创意，从而帮助你赢得机器学习竞赛。
这种易用性并不以降低灵活性为代价：因为 Keras 与底层深度学习语言（特别是 TensorFlow）集成在一起，
所以它可以让你实现任何你可以用基础语言编写的东西。特别是，tf.keras 作为 Keras API 可以与 TensorFlow 工作流无缝集成。
"""
from keras import models
from keras import layers
import numpy as np
import time

"""
后面会反复构造同一种结构的网络，所以我们把网络的构造代码放在一个函数中，
后面只要直接调用该函数就可以将网络迅速初始化
"""
def build_model(d):
    model = models.Sequential()
    model.add(layers.Dense(d, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(d, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model
# 为了方便后续的参数调优，我们需要在训练过程中对多个参数的训练效果进行尝试
def test_build_model():
    # 以遍历训练输出的方式进行（对神经网络节点数以16为起点，16为步长，128为终点进行便利训练）
    for d in range(16, 129, 16):
        print ('d = ',d) 
        # 以遍历训练输出的方式进行（对神经网络梯度更新样本数以16为起点，16为步长，128为终点进行便利训练）
        for b in range(16, 129, 16):
            print ('b = ',b)
            time1 = time.time() # 对每次训练进行计时
            k = 4 # 设置训练集分割数
            num_val_samples = len(train_data) // k 
            num_epochs = 200 # 设置训练轮数
            all_mae_histories = [] # 记录每次训练的误差信息，便于把握模型准确率
            # 分割训练集（训练集+验证集）
            for i in range(k):
                #依次把k分数据中的每一份作为校验数据集
                val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
                val_targets = train_targets[i* num_val_samples : (i+1) * num_val_samples]
                #把剩下的k-1分数据作为训练数据,如果第i分数据作为校验数据，那么把前i-1份和第i份之后的数据连起来
                partial_train_data = np.concatenate([train_data[: i * num_val_samples], 
                                                     train_data[(i+1) * num_val_samples:]], axis = 0)
                partial_train_targets = np.concatenate([train_targets[: i * num_val_samples], 
                                                        train_targets[(i+1) * num_val_samples: ]],
                                                      axis = 0)
                model = build_model(d)
                #把分割好的训练数据和校验数据输入网络
                history = model.fit(partial_train_data, partial_train_targets, 
                          validation_data=(val_data, val_targets),
                          epochs = num_epochs, 
                          batch_size = b, verbose = 0)
                mae_history = history.history['val_mean_absolute_error']
                all_mae_histories.append(mae_history) 
            # 记录每次训练的误差信息，便于把握模型准确率
            average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]
            min_average_mae = min(average_mae_history)
            print ('min_average_mae = ', min_average_mae)
            #找出平均误差最小的轮次
            for i in range(len(average_mae_history)):
                if average_mae_history[i] == min_average_mae:
                    print ('min_numst = ', i)
            time2 = time.time()
            print (u'总共耗时：' + str(time2 - time1) + 's')
time1 = time.time()
time2 = time.time()
test_build_model()

d =  16
b =  16
min_average_mae =  2.3694010323817185
min_numst =  197
总共耗时：92.72838377952576s
b =  32
min_average_mae =  2.3743427327953945
min_numst =  183
总共耗时：52.38618350028992s
b =  48
min_average_mae =  2.4313561022281647
min_numst =  199
总共耗时：39.08164381980896s
b =  64
min_average_mae =  2.438858114256717
min_numst =  199
总共耗时：30.371596813201904s
b =  80
min_average_mae =  2.727403668483885
min_numst =  199
总共耗时：26.616595029830933s
b =  96
min_average_mae =  2.706521441735844
min_numst =  199
总共耗时：27.12059450149536s
b =  112
min_average_mae =  2.8558388352394104
min_numst =  199
总共耗时：22.43609118461609s
b =  128
min_average_mae =  3.0928390622138977
min_numst =  199
总共耗时：22.75259232521057s
d =  32
b =  16
min_average_mae =  2.268304481954858
min_numst =  190
总共耗时：100.16834044456482s
b =  32
min_average_mae =  2.3481622481405147
min_numst =  195
总共耗时：57.970677614212036s
b =  48
min_average_mae =  2.343408205721638
min_numst =  188
总共耗时：44.76966619491577s
b =  64
min_average_mae = 

In [12]:
# 由上面分析可知训练误差最小（2.21）的点在节点数=112，梯度更新样本数=48，训练轮数=135，模型预测准确率最高
# 重新按准确率最高的参数配置训练模型
model = build_model(112)
model.fit(train_data, train_targets, epochs = 135, batch_size = 48, verbose = 0)
# 根据输入的测试数据进行预测
predict_value = model.predict(test_data, batch_size=48, verbose=0)
# 以上输出的predict_value为矩阵类型，需要转化为列表格式，便于对元素进行相关操作
predict_value_list = list(predict_value.ravel())
predict_int_value = []
# 由于预测的数值为浮点型，按照实际情况需要转化为非负整型变量
for value in predict_value_list:
    value = round(value)
    if value<0:
        value = 0
    predict_int_value.append(value)
# 输出测试集中所有的预测值
print ("test predict values:", predict_int_value)

test predict values: [9.0, 20.0, 24.0, 34.0, 27.0, 23.0, 29.0, 23.0, 20.0, 23.0, 21.0, 17.0, 16.0, 44.0, 22.0, 22.0, 28.0, 20.0, 20.0, 27.0, 12.0, 12.0, 23.0, 18.0, 22.0, 25.0, 31.0, 34.0, 12.0, 22.0, 21.0, 16.0, 36.0, 26.0, 20.0, 8.0, 17.0, 16.0, 18.0, 28.0, 35.0, 29.0, 13.0, 46.0, 30.0, 27.0, 29.0, 19.0, 24.0, 24.0, 39.0, 21.0, 11.0, 15.0, 38.0, 30.0, 12.0, 51.0, 37.0, 25.0, 25.0, 16.0, 17.0, 21.0, 25.0, 23.0, 13.0, 24.0, 13.0, 7.0, 26.0, 33.0, 27.0, 13.0, 28.0, 20.0, 21.0, 26.0, 39.0, 10.0, 23.0, 40.0, 18.0, 13.0, 19.0, 19.0, 24.0, 21.0, 23.0, 30.0, 21.0, 21.0, 28.0, 47.0, 40.0, 21.0, 37.0, 47.0, 28.0, 49.0, 33.0, 20.0]
